## Workspace Management demo

This notebook presents examples of how to manage user workspaces.

In [1]:
import requests
import datetime

In [2]:
# base_domain = "demo.eoepca.org"
base_domain = "develop.eoepca.org"
workspaces_endpoint = f'https://workspace-api.{base_domain}/workspaces'

### Creation and retrieval of credentials

The following request will create a user workspace.

In [3]:
preferred_workspace_name = f"my-new-user-{datetime.datetime.now().timestamp():.0f}"
preferred_workspace_name

'my-new-user-1647520959'

In [4]:
response = requests.post(
    workspaces_endpoint,
    json={
        "preferred_name": preferred_workspace_name,
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
response.json()

{'name': 'develop-user-my-new-user-1647520959'}

Workspace creation is an asynchronous process, so when we try to retrieve the workspace data right now, we see that it is still provisioning.

In [5]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()

response.json()

{'status': 'provisioning',
 'endpoints': [],
 'storage': None,
 'container_registry': None}

In [6]:
import time
# wait for provisioning
time.sleep(10)

Now all the credentials should be set up and are available:

In [7]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()

workspace_data = response.json()
workspace_data["storage"]["credentials"]['secret'] = "hidden_for_demo"
workspace_data["container_registry"]["password"] = "hidden_for_demo"
workspace_data

{'status': 'ready',
 'endpoints': [],
 'storage': {'credentials': {'access': 'develop-user-my-new-user-1647520959+c81f6e30-2',
   'bucketname': 'develop-user-my-new-user-1647520959',
   'projectid': 'd346903185d941958475387791512554',
   'secret': 'hidden_for_demo',
   'endpoint': 'https://cf2.cloudferro.com:8080',
   'region': 'RegionOne'}},
 'container_registry': {'username': 'develop-user-my-new-user-1647520959',
  'password': 'hidden_for_demo'}}

## Redeploy

In case something went wrong or an update is necessary, it's possible to trigger the provisioning again:

In [8]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/redeploy",
)
response.raise_for_status()
response

<Response [204]>

## Deletion
Workspaces can also be deleted like this:

In [9]:
response = requests.delete(
    f"{workspaces_endpoint}/{actual_workspace_name}",
)
response.raise_for_status()
response

<Response [204]>